In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str, "SUPERFICIE": str})

In [17]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "COLOR", "TIPO", "MODO"])

In [18]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA","PRODUCCION", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != "0.0")]

to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]
l_encoder = LabelEncoder()
train["SUPERFICIE"] = l_encoder.fit_transform(train["SUPERFICIE"])

to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

C:\Users\adars\AppData\Local\Temp\ipykernel_11060\3463993660.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["SUPERFICIE"] = l_encoder.fit_transform(train["SUPERFICIE"])


In [19]:
df_supcat = train.groupby(by='SUPERFICIE').count().reset_index()
segur_train = df_supcat[df_supcat["TIPO"] == 1]["SUPERFICIE"]
train_nou = train[train['SUPERFICIE'].isin(list(segur_train))]

In [20]:
train_a_splitejar = train.drop(index=train_nou.index)

In [21]:
X_train_a_splitejar = train_a_splitejar.drop(axis = 1, columns = ["SUPERFICIE"])
y_train_a_splitejar = train_a_splitejar["SUPERFICIE"]

In [22]:
X_train_nou = train_nou.drop(axis = 1, columns = ["SUPERFICIE"])
y_train_nou = train_nou["SUPERFICIE"]

In [50]:
X_train_splitted, X_test, y_train_splitted, y_test = train_test_split(X_train_a_splitejar, y_train_a_splitejar, test_size=0.35, random_state=3, stratify = train_a_splitejar["SUPERFICIE"])

In [24]:
X_train = pd.concat([X_train_splitted, X_train_nou])
y_train = pd.concat([y_train_splitted, y_train_nou])

In [25]:
len(X_test) + len(X_train) == len(train)

True

In [28]:
import xgboost as xgb
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 100, 'min_child_weight': 1, 'n_estimators': 2000}
model = xgb.XGBClassifier(**params_model)
model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=100, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=2000, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [29]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.7248182762201454
0.7334905660377359


In [30]:
y_test_dec_gt = l_encoder.inverse_transform(y_test)
y_predicted = l_encoder.inverse_transform(model.predict(X_test))

In [31]:
y_test_dec_gt = y_test_dec_gt.astype(np.float)
y_predicted = y_predicted.astype(np.float)

C:\Users\adars\AppData\Local\Temp\ipykernel_11060\2686677544.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_test_dec_gt = y_test_dec_gt.astype(np.float)
C:\Users\adars\AppData\Local\Temp\ipykernel_11060\2686677544.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_predicted = y_predicted.astype(np.float)


In [32]:
from sklearn.metrics import r2_score
r2_score(y_test_dec_gt, y_predicted)

0.43315455142273485

In [202]:
len(y_train)

2120

In [204]:
model.predict_proba(X_test)

array([[6.8190414e-04, 3.8513844e-04, 1.7557872e-04, ..., 5.1132516e-05,
        1.1260376e-04, 5.8873522e-04],
       [1.4687682e-04, 4.1054318e-05, 9.5589610e-05, ..., 1.7975106e-04,
        3.6124216e-05, 8.1145583e-05],
       [3.8297463e-04, 1.6848360e-04, 1.4259036e-04, ..., 2.7710092e-04,
        1.7916671e-03, 4.3455581e-04],
       ...,
       [1.2515013e-03, 3.2755014e-04, 1.6364275e-04, ..., 1.5561598e-04,
        5.9414026e-04, 2.9936555e-04],
       [2.1049769e-04, 5.8837293e-05, 2.6087352e-04, ..., 2.5761157e-04,
        5.1771684e-05, 1.1629438e-04],
       [1.4517110e-04, 1.2535493e-04, 3.9822314e-04, ..., 2.7865608e-04,
        1.0341343e-04, 4.9287215e-04]], dtype=float32)

In [43]:
X_test = X_test.reset_index()

In [51]:
X_test

,ID_FINCA,ID_ZONA,ID_ESTACION,VARIEDAD,MODO,TIPO,COLOR
8200,580,68,8,4,2,1,1
9223,541,65,14,9,2,1,2
7875,284,43,11,5,2,1,1
8166,529,65,14,7,2,1,2
8304,703,73,14,8,1,1,1
...,...,...,...,...,...,...,...
7691,1064,29,9,13,1,1,2
9312,638,72,14,2,1,1,1
8071,424,57,10,2,1,1,1
7333,803,74,13,9,2,1,2


In [78]:
llista_idx = np.where(np.array(y_test) == np.array(model.predict(X_test)))[0]

In [59]:
len(np.array(model.predict(X_test)))

963

In [74]:
y_train.nunique()

1211

In [75]:
X_test.iloc[0, :]

ID_FINCA       580
ID_ZONA         68
ID_ESTACION      8
VARIEDAD         4
MODO             2
TIPO             1
COLOR            1
Name: 8200, dtype: int32

In [89]:
model.predict_proba(X_test.iloc[llista_idx, :])[0]

array([6.8740122e-04, 3.8630271e-04, 1.7607113e-04, ..., 5.1061033e-05,
       1.1203519e-04, 5.9812365e-04], dtype=float32)

In [94]:
llista_probs = model.predict_proba(X_test.iloc[llista_idx, :])

In [98]:
np.max(llista_probs[1])

0.7917654

In [103]:
prob_max = np.array([np.max(llista_probs[i]) for i in range(len(llista_idx))])

In [106]:
len(prob_max)

698

In [117]:
dict_prob = {"Menor a 0.5": (prob_max < 0.5).sum()/len(prob_max), "Major a 0.5": (prob_max > 0.5).sum()/len(prob_max),
             "Major a 0.6": (prob_max > 0.6).sum()/len(prob_max), "Major a 0.65": (prob_max > 0.65).sum()/len(prob_max),
            "Major a 0.75": (prob_max > 0.75).sum()/len(prob_max), "Major a 0.8": (prob_max > 0.8).sum()/len(prob_max),
            "Major a 0.9": (prob_max > 0.9).sum()/len(prob_max), "Major a 0.95": (prob_max > 0.95).sum()/len(prob_max)}
dict_counts = {"Menor a 0.5": (prob_max < 0.5).sum(), "Major a 0.5": (prob_max > 0.5).sum(),
               "Major a 0.6": (prob_max > 0.6).sum(), "Major a 0.65": (prob_max > 0.65).sum(),
            "Major a 0.75": (prob_max > 0.75).sum(), "Major a 0.8": (prob_max > 0.8).sum(),
            "Major a 0.9": (prob_max > 0.9).sum(), "Major a 0.95": (prob_max > 0.95).sum()}

In [118]:
df_prob = pd.DataFrame(dict_prob, index=[0])
df_counts = pd.DataFrame(dict_counts, index=[0])

In [119]:
df_prob

,Menor a 0.5,Major a 0.5,Major a 0.6,Major a 0.65,Major a 0.75,Major a 0.8,Major a 0.9,Major a 0.95
0,0.266476,0.733524,0.232092,0.19341,0.103152,0.055874,0.018625,0.0


In [120]:
df_counts

,Menor a 0.5,Major a 0.5,Major a 0.6,Major a 0.65,Major a 0.75,Major a 0.8,Major a 0.9,Major a 0.95
0,186,512,162,135,72,39,13,0


In [130]:
(prob_max > 0.4).sum()

639

In [129]:
len(y_test)

963

In [125]:
((0.45 <= prob_max) & (prob_max <= 0.65)).sum()

456

In [126]:
#params_model2 = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 100, 'min_child_weight': 1, 'n_estimators': 600}
param2 = {}
param2['booster'] = 'gbtree'
param2["eval_metric"] = "error"
param2['eta'] = 0.3
param2['gamma'] = 0
param2['max_depth'] = 6
param2['min_child_weight']=1
param2['max_delta_step'] = 0
param2['subsample']= 1
param2['colsample_bytree']=1
param2['seed'] = 0
param2['base_score'] = 0.5
param2["colsample_bytree"] = 0.4
param2["learning_rate"] = 0.1
param2["n_estimators"] = 2000

model2 = xgb.XGBClassifier(**param2)
model2.fit(X_train,y_train)

[23:26:26] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.



XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, early_stopping_rounds=None,
              enable_categorical=False, eta=0.3, eval_metric='error',
              feature_types=None, gamma=0, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=0, max_depth=6,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=2000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [127]:
print(model2.score(X_test, y_test))
print(model2.score(X_train, y_train))

0.726895119418484
0.7344339622641509


In [131]:
llista_probs = model.predict_proba(X_test)

In [132]:
llista_probs[0]

array([6.8740122e-04, 3.8630271e-04, 1.7607113e-04, ..., 5.1061033e-05,
       1.1203519e-04, 5.9812365e-04], dtype=float32)

In [156]:
thr_04 = [idx for idx,i in enumerate(llista_probs) if np.max(i) > 0.5]

In [157]:
X_test_subset = X_test.iloc[thr_04, :]

In [158]:
(np.array(y_test.iloc[thr_04]) == np.array(model.predict(X_test_subset))*1).sum()

512

In [159]:
len(y_test.iloc[thr_04])

521